# Computation on Arrays: Broadcasting

We saw in the previous section how NumPy's universal functions can be used to *vectorize* operations and thereby remove slow Python loops.
Another means of vectorizing operations is to use NumPy's *broadcasting* functionality.
Broadcasting is simply a set of rules for applying binary ufuncs (e.g., addition, subtraction, multiplication, etc.) on arrays of different sizes.

## Introducing Broadcasting

Recall that for arrays of the same size, binary operations are performed on an element-by-element basis:

In [1]:
import numpy as np

In [3]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])

In [4]:
a

array([0, 1, 2])

In [5]:
b

array([5, 5, 5])

In [10]:
a + b

array([5, 6, 7])

Broadcasting allows these types of binary operations to be performed on arrays of different sizes–for example, we can just as easily add a scalar (think of it as a zero-dimensional array) to an array:

In [7]:
a

array([0, 1, 2])

In [8]:
5

5

In [9]:
a + 5

array([5, 6, 7])

We can think of this as an operation that stretches or duplicates the value ``5`` into the array ``[5, 5, 5]``, and adds the results.
The advantage of NumPy's broadcasting is that this duplication of values does not actually take place, but it is a useful mental model as we think about broadcasting.

We can similarly extend this to arrays of higher dimension. Observe the result when we add a one-dimensional array to a two-dimensional array:

In [12]:
M = np.ones((3, 3))
M

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [13]:
a

array([0, 1, 2])

In [14]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

Here the one-dimensional array ``a`` is stretched, or broadcast across the second dimension in order to match the shape of ``M``.

While these examples are relatively easy to understand, more complicated cases can involve broadcasting of both arrays. Consider the following example:

In [15]:
a = np.arange(3)
b = np.arange(3)[:, np.newaxis]

print(a)
print()
print(b)

[0 1 2]

[[0]
 [1]
 [2]]


In [16]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

Just as before we stretched or broadcasted one value to match the shape of the other, here we've stretched *both* ``a`` and ``b`` to match a common shape, and the result is a two-dimensional array!
The geometry of these examples is visualized in the following figure (Code to produce this plot can be found in the [appendix](06.00-Figure-Code.ipynb#Broadcasting), and is adapted from source published in the [astroML](http://astroml.org) documentation. Used by permission).

![Broadcasting Visual](figures/02.05-broadcasting.png)

The light boxes represent the broadcasted values: again, this extra memory is not actually allocated in the course of the operation, but it can be useful conceptually to imagine that it is.

## Rules of Broadcasting

Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

- Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is *padded* with ones on its leading (left) side.
- Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
- Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

To make these rules clear, let's consider a few examples in detail.

### Broadcasting example 1

Let's look at adding a two-dimensional array to a one-dimensional array:

In [17]:
M = np.ones((2, 3))
a = np.arange(3)

In [19]:
print('M:', M)
print()
print('a:', a)

M: [[1. 1. 1.]
 [1. 1. 1.]]

a: [0 1 2]


In [24]:
M

array([[1., 1., 1.],
       [1., 1., 1.]])

In [25]:
a

array([0, 1, 2])

In [26]:
M.shape

(2, 3)

In [27]:
a.shape

(3,)

In [28]:
a = a.reshape(1,3)
a.shape

(1, 3)

Let's consider an operation on these two arrays. The shape of the arrays are

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``

We see by rule 1 that the array ``a`` has fewer dimensions, so we pad it on the left with ones:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

By rule 2, we now see that the first dimension disagrees, so we stretch this dimension to match:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

The shapes match, and we see that the final shape will be ``(2, 3)``:

In [29]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### Broadcasting example 2

Let's take a look at an example where both arrays need to be broadcast:

In [38]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

In [39]:
a

array([[0],
       [1],
       [2]])

In [40]:
a.shape

(3, 1)

In [41]:
b

array([0, 1, 2])

In [42]:
b.shape

(3,)

In [25]:
b = b.reshape(1,3)

Again, we'll start by writing out the shape of the arrays:

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

Rule 1 says we must pad the shape of ``b`` with ones:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

And rule 2 tells us that we upgrade each of these ones to match the corresponding size of the other array:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

Because the result matches, these shapes are compatible. We can see this here:

In [43]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Broadcasting example 3

Now let's take a look at an example in which the two arrays are not compatible:

In [60]:
M = np.ones((3, 2))
a = np.arange(3)

In [61]:
M

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [62]:
a

array([0, 1, 2])

In [45]:
M.shape

(3, 2)

In [46]:
a.shape

(3,)

In [47]:
a = a.reshape(1,3)

This is just a slightly different situation than in the first example: the matrix ``M`` is transposed.
How does this affect the calculation? The shape of the arrays are

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

Again, rule 1 tells us that we must pad the shape of ``a`` with ones:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

By rule 2, the first dimension of ``a`` is stretched to match that of ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

Now we hit rule 3–the final shapes do not match, so these two arrays are incompatible, as we can observe by attempting this operation:

In [59]:
M + a

ValueError: operands could not be broadcast together with shapes (3,2) (1,3) 

Note the potential confusion here: you could imagine making ``a`` and ``M`` compatible by, say, padding ``a``'s shape with ones on the right rather than the left.
But this is not how the broadcasting rules work!
That sort of flexibility might be useful in some cases, but it would lead to potential areas of ambiguity.
If right-side padding is what you'd like, you can do this explicitly by reshaping the array (we'll use the ``np.newaxis`` keyword introduced in [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb)):

In [49]:
a[:, np.newaxis].shape

(1, 1, 3)

In [50]:
M + a[:, np.newaxis]

ValueError: operands could not be broadcast together with shapes (3,2) (1,1,3) 

Also note that while we've been focusing on the ``+`` operator here, these broadcasting rules apply to *any* binary ``ufunc``.
For example, here is the ``logaddexp(a, b)`` function, which computes ``log(exp(a) + exp(b))`` with more precision than the naive approach:

In [51]:
np.logaddexp(M, a[:, np.newaxis])

ValueError: operands could not be broadcast together with shapes (3,2) (1,1,3) 

For more information on the many available universal functions, refer to [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb).

## Broadcasting in Practice

Broadcasting operations form the core of many examples we'll see throughout this book.
We'll now take a look at a couple simple examples of where they can be useful.

### Centering an array

In the previous section, we saw that ufuncs allow a NumPy user to remove the need to explicitly write slow Python loops. Broadcasting extends this ability.
One commonly seen example is when centering an array of data.
Imagine you have an array of 10 observations, each of which consists of 3 values.
Using the standard convention (see [Data Representation in Scikit-Learn](05.02-Introducing-Scikit-Learn.ipynb#Data-Representation-in-Scikit-Learn)), we'll store this in a $10 \times 3$ array:

In [52]:
X = np.random.random((10, 3))
X

array([[0.98985198, 0.39597228, 0.62979126],
       [0.41771722, 0.92429046, 0.4584903 ],
       [0.77024741, 0.90503116, 0.18318218],
       [0.34318092, 0.4656394 , 0.7294469 ],
       [0.60040069, 0.80859266, 0.09793124],
       [0.9568181 , 0.4486861 , 0.72104129],
       [0.97630489, 0.70068237, 0.41745046],
       [0.84283577, 0.2558336 , 0.55449979],
       [0.27642728, 0.01991348, 0.17115748],
       [0.15281858, 0.4427479 , 0.9134453 ]])

We can compute the mean of each feature using the ``mean`` aggregate across the first dimension:

In [53]:
Xmean = X.mean(0)
Xmean

array([0.63266028, 0.53673894, 0.48764362])

And now we can center the ``X`` array by subtracting the mean (this is a broadcasting operation):

In [58]:
X_centered = X - Xmean
X_centered

array([[ 0.3571917 , -0.14076666,  0.14214764],
       [-0.21494307,  0.38755152, -0.02915332],
       [ 0.13758713,  0.36829221, -0.30446144],
       [-0.28947937, -0.07109954,  0.24180328],
       [-0.03225959,  0.27185372, -0.38971238],
       [ 0.32415782, -0.08805284,  0.23339767],
       [ 0.34364461,  0.16394343, -0.07019316],
       [ 0.21017548, -0.28090534,  0.06685617],
       [-0.35623301, -0.51682546, -0.31648614],
       [-0.4798417 , -0.09399104,  0.42580168]])

To double-check that we've done this correctly, we can check that the centered array has near zero mean:

In [56]:
X_centered.mean(0)

array([-9.99200722e-17,  6.66133815e-17,  0.00000000e+00])

To within machine precision, the mean is now zero.

In [63]:
import numpy as np
x1=np.array([1, 2, 3])
val=x1+10
print('broadcast add 10', val)

broadcast add 10 [11 12 13]


In [64]:
#Basic 2
# Create a 2D array
arr = np.array([[1, 2], [3, 4]])

# Multiply the array by a scalar value of 2 using broadcasting
arr_2 = arr * 2

print("2D Array after multiplying by scalar 2:\n", arr_2)

2D Array after multiplying by scalar 2:
 [[2 4]
 [6 8]]


In [66]:
import numpy as np

array_3d = np.array([[[1, 2], [3, 4]], 
                     [[5, 6], [7, 8]]])

array_2d = np.array([[10, 20], [30, 40]])

#broadcasting
result = array_3d - array_2d[:, :, np.newaxis]

print("3D Array:\n", array_3d)
print("\n2D Array:\n", array_2d)
print("\nResult after subtraction using broadcasting:\n", result)

3D Array:
 [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]

2D Array:
 [[10 20]
 [30 40]]

Result after subtraction using broadcasting:
 [[[ -9  -8]
  [-17 -16]]

 [[-25 -24]
  [-33 -32]]]


In [67]:
#Intermediate 1
# Create a 2D array and a 1D array
arr_2d = np.array([[1, 2, 3], [4, 5, 6]])
arr_1d = np.array([10, 20, 30])

# Add the 1D array to the 2D array using broadcasting
arr_sum = arr_2d + arr_1d

print("2D Array after adding 1D array using broadcasting:\n", arr_sum)

2D Array after adding 1D array using broadcasting:
 [[11 22 33]
 [14 25 36]]


In [68]:
 array_1d = np.array([1, 2, 3])
print("Original 1D array:", array_1d)

result_1d = array_1d + 10
print("1D array after adding 10:", result_1d)

array_2d = np.array([[1, 2], [3, 4]])
print("\nOriginal 2D array:")
print(array_2d)

result_2d = array_2d * 2
print("2D array after multiplying by 2:")
print(result_2d)

Original 1D array: [1 2 3]
1D array after adding 10: [11 12 13]

Original 2D array:
[[1 2]
 [3 4]]
2D array after multiplying by 2:
[[2 4]
 [6 8]]


In [69]:
import numpy as np
array1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sum_greater_than_5 = np.sum(array1[array1 > 5])
mean_less_than_equal_5 = np.mean(array1[array1 <= 5])

array2 = np.array([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
max_along_columns = np.max(array2, axis=0)
product_all_elements = np.prod(array2)
print(f"Sum of elements greater than 5: {sum_greater_than_5}")
print(f"Mean of elements less than or equal to 5: {mean_less_than_equal_5}")
print(f"Maximum value along columns: {max_along_columns}")
print(f"Product of all elements: {product_all_elements}")

Sum of elements greater than 5: 30
Mean of elements less than or equal to 5: 3.0
Maximum value along columns: [70 80 90]
Product of all elements: -1786839040


In [70]:
#Advanced 1
# Create a 3D array and a 2D array
arr_3d = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
arr_2d = np.array([[10, 20], [30, 40]])

# Subtract the 2D array from the 3D array using broadcasting
arr_subtracted = arr_3d - arr_2d

print("3D Array after subtracting the 2D array using broadcasting:\n", arr_subtracted)

3D Array after subtracting the 2D array using broadcasting:
 [[[ -9 -18]
  [-27 -36]]

 [[ -5 -14]
  [-23 -32]]]


In [71]:
array2d = np.array([[1, 2, 3], [4, 5, 6]])
array1d = np.array([10, 20, 30])
array2d + array1d

array([[11, 22, 33],
       [14, 25, 36]])

In [ ]:
>>> a_3d=np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
>>> b_2d=np.array([[10, 20], [30, 40]])
>>> print(a_3d)
[[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]
>>> print(b_2d)
[[10 20]
 [30 40]]
>>> 
>>> 
>>> np.subtract(a_3d, b_2d)
array([[[ -9, -18],
        [-27, -36]],

       [[ -5, -14],
        [-23, -32]]])
>>>

In [73]:
x_2d=np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(x_2d, '\n')

x_2d[x_2d>5]=0
print(x_2d)

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[1 2 3]
 [4 5 0]
 [0 0 0]]


In [74]:
#Advance 1
import numpy as np

# Create a 2D array 
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Use a boolean array to set all elements greater than 5 to 0
arr[arr > 5] = 0

# Print the modified array
print("Modified array:\n", arr)

Modified array:
 [[1 2 3]
 [4 5 0]
 [0 0 0]]
